In [13]:
import os
PROJECT_DIR = os.path.abspath(os.path.join("..", ".."))
DATA_DIR = os.path.join(PROJECT_DIR, "data")
DB_DIR = os.path.join(PROJECT_DIR, "backend", "chroma_db")

LLM_MODEL = "llama3.2"
EMBED_MODEL = "nomic-embed-text"


In [14]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader


In [15]:
def load_docs():
    docs=[]
    for fn in os.listdir(DATA_DIR):
        p=os.path.join(DATA_DIR, fn)
        if fn.lower().endswith(".pdf"):
            docs += PyPDFLoader(p).load()
        elif fn.lower().endswith((".txt",".md")):
            docs += TextLoader(p, encoding="utf-8").load()
    return docs

docs = load_docs()
len(docs), docs[0].metadata if docs else None


(1,
 {'source': 'c:\\Graduation\\Agentic AI\\Practice\\rag-voice-chatbot\\data\\test.txt'})

In [16]:
vs = Chroma.from_documents(
    documents=chunks,
    embedding=emb,
    persist_directory=DB_DIR
)

retriever = vs.as_retriever(search_kwargs={"k": 4})


In [17]:
llm = OllamaLLM(model=LLM_MODEL, temperature=0.2)

def rag(q):
    rel = retriever.invoke(q)
    ctx = "\n\n".join([d.page_content for d in rel])
    prompt = f"Use only context.\n\nCONTEXT:\n{ctx}\n\nQ:{q}\nA:"
    return llm.invoke(prompt), rel

ans, rel = rag("مين اللي بيبني RAG chatbot؟")
ans[:800], rel[0].metadata if rel else None


('أنا بني RAG chatbot.',
 {'source': 'c:\\Graduation\\Agentic AI\\Practice\\rag-voice-chatbot\\data\\test.txt'})